In [173]:
import pandas as pd

In [174]:
import os

In [175]:
%pwd

'/Users/vallirajasekar/Desktop/ML/Red_Wine_Quality/End-to-End-ML-Project'

In [176]:
os.chdir("/Users/vallirajasekar/Desktop/ML/Red_Wine_Quality/End-to-End-ML-Project")

In [177]:
%pwd

'/Users/vallirajasekar/Desktop/ML/Red_Wine_Quality/End-to-End-ML-Project'

## Creation of Entity 

In [178]:
## Create the Entity -- Return type of Function

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## Configuration manager

In [179]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [180]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        
        if 'Id' in config:
            del config['Id']

        return data_ingestion_config

In [181]:
#schema

## Data Ingestion Components

In [182]:
import os
import urllib.request as request
import zipfile
#from src.mlProject import logger
from src.mlProject.utils.common import get_size
#import logging  # Importing logging module directly
from src.mlProject.logging import *
from src.mlProject.constants import *


In [183]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


## Pipeline

In [184]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-30 14:33:53,216: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-30 14:33:53,218: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-30 14:33:53,220: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-30 14:33:53,221: INFO: common: created directory at: artifacts]
[2024-03-30 14:33:53,222: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-30 14:33:55,126: INFO: 1268487314: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 22234
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "cff2ac1407ee6cd842e8a930d18e44c9b4b704357ed9ccffdff77e8bca49a4d9"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DC8A:7E59E:698BA:98BDC:6607C78E
Accept-Ranges: bytes
Date: Sat, 30 Ma

In [185]:
data_ingestion

In [186]:
data_ingestion_config

DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://github.com/vallirajasekar/Branching_tutorial/raw/main/WineQT.csv.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')

In [187]:
config